<a href="https://colab.research.google.com/github/JITHIN-ANTONY-JOSEPH/ERP_11358080/blob/main/10_Experiment9_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Input : Recipe Instructions
### Model : Word2Vec(Text) + Node2Vec(Graph) + Attention based Neural Networks(Model) with Negative Sampling -> V1(BATCH SIZE = 128)

### Mounting to connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importing required libraries , loading datasets and pre-processing

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
import random
import jellyfish
from gensim.models import Word2Vec
import pickle
import networkx as nx
import json

# Load the main dataset (modify the path as per your setup)
with open('/content/drive/My Drive/ERP/modified_Processed_Layer1.json', 'r') as file: # Adjust the path as needed , this is the path to my personal Google Drive
    recipe1m_data = [json.loads(line) for line in file]
recipe1m_df = pd.DataFrame(recipe1m_data)

# Load the substitution pairs (modify the path as per your setup)
substitution_pairs_df = pd.read_csv('/content/drive/My Drive/ERP/Recipe1MSubs_full.csv') # Adjust the path as needed , this is the path to my personal Google Drive

# Load flavor graph nodes (modify the path as per your setup)
flavorgraph_df = pd.read_csv('/content/drive/My Drive/ERP/Dataset/nodes_191120.csv') # Adjust the path as needed , this is the path to my personal Google Drive

# Load the precomputed graph embeddings
with open('/content/drive/My Drive/ERP/MODEL_BEST_NUMBERS/graph_embeddings.pkl', 'rb') as f: # Adjust the path as needed , this is the path to my personal Google Drive
    graph_embeddings = pickle.load(f)

# Ingredient list for NER-like extraction
ingredient_list = set(flavorgraph_df[flavorgraph_df['node_type'] == 'ingredient']['name'].dropna().unique())

# Function to extract ingredients from instructions
def extract_ingredients_from_instructions(instructions, ingredient_list):
    extracted_ingredients = []
    for instruction in instructions:
        words = instruction.split()
        for word in words:
            if word in ingredient_list:
                extracted_ingredients.append(word)
    return extracted_ingredients

# Apply the extraction function
recipe1m_df['extracted_ingredients'] = recipe1m_df['processed_instructions'].apply(
    lambda instructions: extract_ingredients_from_instructions(instructions, ingredient_list) if isinstance(instructions, list) else []
)

# Prepare sentences for training the Word2Vec model
sentences = recipe1m_df['extracted_ingredients'].tolist()

# Add substitution contexts to sentences
for _, row in substitution_pairs_df.iterrows():
    ingredient1 = row['ingredient1']
    ingredient2 = row['ingredient2']
    sentences.append([ingredient1, ingredient2])

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=8)

# Generate valid substitutes dictionary
valid_substitutes = {}
for _, row in substitution_pairs_df.iterrows():
    ing1 = row['ingredient1']
    ing2 = row['ingredient2']
    if ing1 not in valid_substitutes:
        valid_substitutes[ing1] = set()
    valid_substitutes[ing1].add(ing2)


### Function to generate combined emedding

In [ ]:
def get_combined_embedding(ingredient, text_embeddings, graph_embeddings):
    # Get text embedding
    if ingredient in text_embeddings:
        text_embedding = text_embeddings[ingredient]
    else:
        text_embedding = np.zeros(100)

    # Get graph embedding
    if ingredient in graph_embeddings:
        graph_embedding = graph_embeddings[ingredient]
    else:
        graph_embedding = np.zeros(100)

    # Combine embeddings by concatenation
    combined_embedding = np.concatenate((text_embedding, graph_embedding))

    return combined_embedding

In [ ]:
# Prepare training data with negative sampling
train_data = []
train_labels = []
negative_samples = []
negative_labels = []

for _, row in substitution_pairs_df.iterrows():
    ing1 = row['ingredient1']
    ing2 = row['ingredient2']

    combined_embedding1 = get_combined_embedding(ing1, model.wv, graph_embeddings)
    combined_embedding2 = get_combined_embedding(ing2, model.wv, graph_embeddings)

    train_data.append(combined_embedding1)
    train_labels.append(combined_embedding2)

    # Generate negative samples by excluding valid substitutes
    possible_negatives = [ing for ing in ingredient_list if ing != ing1 and ing not in valid_substitutes.get(ing1, set())]
    selected_negatives = random.sample(possible_negatives, min(100, len(possible_negatives)))

    for neg in selected_negatives:
        neg_embedding = get_combined_embedding(neg, model.wv, graph_embeddings)
        negative_labels.append(neg_embedding)
        negative_samples.append(combined_embedding1)

### Defining the attention based neural network and Combined Model run

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
import json
import jellyfish
import random

# Define the neural network with attention
class AttentionLayer(nn.Module):
    def __init__(self, input_dim, attention_dim):
        super(AttentionLayer, self).__init__()
        self.fc1 = nn.Linear(input_dim, attention_dim)
        self.fc2 = nn.Linear(attention_dim, 1)

    def forward(self, x):
        # Compute attention scores
        attention_scores = torch.tanh(self.fc1(x))
        attention_scores = self.fc2(attention_scores).squeeze(-1)
        attention_weights = F.softmax(attention_scores, dim=-1)

        # Apply attention weights to input
        weighted_sum = torch.sum(attention_weights.unsqueeze(-1) * x, dim=1)
        return weighted_sum

class CombinedNN(nn.Module):
    def __init__(self, input_dim, output_dim, attention_dim=64):
        super(CombinedNN, self).__init__()
        self.attention_layer = AttentionLayer(input_dim, attention_dim)
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Apply attention
        if x.dim() == 2:  # Ensure x is 3D: [batch_size, sequence_length, input_dim]
            x = x.unsqueeze(1)
        x = self.attention_layer(x)

        # Feed-forward layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Convert to tensors
train_data = torch.tensor(np.array(train_data), dtype=torch.float32)
train_labels = torch.tensor(np.array(train_labels), dtype=torch.float32)
negative_samples = torch.tensor(np.array(negative_samples), dtype=torch.float32)
negative_labels = torch.tensor(np.array(negative_labels), dtype=torch.float32)

# Instantiate the model
nn_model = CombinedNN(input_dim=200, output_dim=200)  # Combined embedding dimension is 200 (100 + 100)
criterion = nn.MSELoss()
optimizer = optim.Adam(nn_model.parameters(), lr=0.001)

# Training loop with negative sampling
batch_size = 128  # Define your batch size here
num_epochs = 100  # Number of epochs

for epoch in range(num_epochs):
    nn_model.train()
    permutation = torch.randperm(train_data.size(0))

    for i in range(0, train_data.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_data = train_data[indices]
        batch_labels = train_labels[indices]

        optimizer.zero_grad()

        # Forward pass
        outputs = nn_model(batch_data)
        loss = criterion(outputs, batch_labels)

        # Negative pair loss (contrastive or margin-based loss)
        neg_indices = permutation[i:i + batch_size]
        batch_neg_samples = negative_samples[neg_indices]
        neg_outputs = nn_model(batch_neg_samples)

        expanded_outputs = outputs.repeat_interleave(len(neg_indices) // len(indices), dim=0)
        negative_loss = torch.mean(torch.relu(1.0 - torch.sum(expanded_outputs * neg_outputs, dim=1)))

        total_loss = loss + negative_loss
        total_loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss.item()}')

# Validation process as per previous code
# Function to calculate cosine similarity and rank top substitutes, etc.
# Calculate metrics for validation


Epoch 1/100, Loss: 0.6873063445091248
Epoch 2/100, Loss: 0.6528133749961853
Epoch 3/100, Loss: 0.7139190435409546
Epoch 4/100, Loss: 0.6652897596359253
Epoch 5/100, Loss: 0.6944208145141602
Epoch 6/100, Loss: 0.6522852778434753
Epoch 7/100, Loss: 0.5993839502334595
Epoch 8/100, Loss: 0.6662512421607971
Epoch 9/100, Loss: 0.6188412308692932
Epoch 10/100, Loss: 0.6447959542274475
Epoch 11/100, Loss: 0.6572091579437256
Epoch 12/100, Loss: 0.6761915683746338
Epoch 13/100, Loss: 0.6150991916656494
Epoch 14/100, Loss: 0.6020552515983582
Epoch 15/100, Loss: 0.6164640784263611
Epoch 16/100, Loss: 0.6408690214157104
Epoch 17/100, Loss: 0.6380998492240906
Epoch 18/100, Loss: 0.6558237075805664
Epoch 19/100, Loss: 0.5994770526885986
Epoch 20/100, Loss: 0.686007559299469
Epoch 21/100, Loss: 0.641018807888031
Epoch 22/100, Loss: 0.6269079446792603
Epoch 23/100, Loss: 0.7093823552131653
Epoch 24/100, Loss: 0.6532923579216003
Epoch 25/100, Loss: 0.6271932721138
Epoch 26/100, Loss: 0.6600909233093262


### Evaluation

In [ ]:
# Validation process
val_data = []
val_labels = []

for _, row in substitution_pairs_df.iterrows():
    if len(val_data) >= 500:
        break
    ing1 = row['ingredient1']
    ing2 = row['ingredient2']  # Ground truth label for validation

    combined_embedding = get_combined_embedding(ing1, model.wv, graph_embeddings)
    val_data.append(combined_embedding)
    val_labels.append(ing2)

val_data = torch.tensor(np.array(val_data), dtype=torch.float32)
nn_model.eval()

# Function to find the top N most similar ingredients based on cosine similarity
def find_top_similar_ingredients(predicted_embedding, combined_embeddings, top_n=10):
    similarities = {}
    for ingredient, embedding in combined_embeddings.items():
        similarity = cosine_similarity(predicted_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarities[ingredient] = similarity
    sorted_ingredients = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    return [ingredient for ingredient, similarity in sorted_ingredients[:top_n]]

# Function to calculate metrics with Jaro-Winkler similarity threshold
def calculate_metrics(predictions, ground_truths, combined_embeddings, top_n=10, threshold=0.8):
    mrr, hit_1, hit_3, hit_10 = 0.0, 0.0, 0.0, 0.0
    total = len(ground_truths)

    for pred, gt in zip(predictions, ground_truths):
        top_similar = find_top_similar_ingredients(pred, combined_embeddings, top_n=top_n)
        for rank, candidate in enumerate(top_similar, start=1):
            sim = jellyfish.jaro_winkler_similarity(gt, candidate)
            if sim >= threshold:
                mrr += 1.0 / rank
                if rank == 1:
                    hit_1 += 1.0
                if rank <= 3:
                    hit_3 += 1.0
                if rank <= 10:
                    hit_10 += 1.0
                break

    mrr /= total
    hit_1 /= total
    hit_3 /= total
    hit_10 /= total
    return mrr, hit_1, hit_3, hit_10

# Final evaluation
with torch.no_grad():
    val_predictions = nn_model(val_data).detach().numpy()
combined_embeddings = {ingredient: get_combined_embedding(ingredient, model.wv, graph_embeddings) for ingredient in model.wv.index_to_key}

mrr, hit_1, hit_3, hit_10 = calculate_metrics(val_predictions, val_labels, combined_embeddings)

print(f"MRR: {mrr:.4f}, Hit@1: {hit_1:.4f}, Hit@3: {hit_3:.4f}, Hit@10: {hit_10:.4f}")


MRR: 0.1675, Hit@1: 0.1060, Hit@3: 0.2000, Hit@10: 0.3140
